# Fine tune CDLA pretrained model for KR court decisions

In [2]:
import subprocess, os, time, json
import fitz
import paddle
paddle.utils.run_check()

!export CUDA_VISIBLE_DEVICES=0

print(os.getcwd())

Running verify PaddlePaddle program ... 


I0524 09:51:30.216377 848456 program_interpreter.cc:212] New Executor is Running.
W0524 09:51:30.218133 848456 gpu_resources.cc:119] Please NOTE: device: 0, GPU Compute Capability: 7.5, Driver API Version: 12.4, Runtime API Version: 11.8
W0524 09:51:30.224120 848456 gpu_resources.cc:164] device: 0, cuDNN Version: 8.8.
I0524 09:51:30.677415 848456 interpreter_util.cc:624] Standalone Executor is Used.


PaddlePaddle works well on 1 GPU.
PaddlePaddle is installed successfully! Let's start deep learning with PaddlePaddle now.
/bin/bash: /home/lstm/miniconda3/envs/pp261/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/home/lstm/github/PaddleDetection/tuning


## Train

In [ ]:
# Single GPU
command = (
    "python3 ../tools/train.py",
    "-c ../configs/picodet/legacy_model/application/layout_analysis/krcase_layout_train_2_5.yml",
    # "--slim_config ../configs/picodet/legacy_model/application/layout_analysis/krcase_layout_distill.yml",
    "--eval",
    f"--use_vdl=True --vdl_log_dir=vdl_log/{time.strftime('%Y%m%d-%H%M%S')}",
    )
command = " ".join(command)
print(command)

vdl_path = command.split('--vdl_log_dir=vdl_log/')[1]

In [ ]:
!{command}
# output = subprocess.run(command, shell=True, capture_output=True)

In [ ]:
# run visualdl in shell (from ./tuning dir)
# visualdl --logdir ./vdl_log/0240520-170427 --port 3001

# vdlcommand = f'visualdl --logdir ./vdl_log/{vdl_path} --port 3001'
# print(vdlcommand)

## Tune

In [2]:
checkpoint = './trained_weights/output_20240503/best_model/model'
# Single GPU
command = (
    "python3 ../tools/train.py",
    "-c ../configs/picodet/legacy_model/application/layout_analysis/krcase_layout_train_tune_2_5.yml",
    # "--slim_config ../configs/picodet/legacy_model/application/layout_analysis/krcase_layout_distill.yml",
    "--eval",
    f"--use_vdl=True --vdl_log_dir=vdl_log/{time.strftime('%Y%m%d-%H%M%S')}",
    )
command = " ".join(command)
print(command)

python3 ../tools/train.py -c ../configs/picodet/legacy_model/application/layout_analysis/krcase_layout_train_tune_2_5.yml --eval --use_vdl=True --vdl_log_dir=vdl_log/20240524-023837


In [3]:
!{command}

[05/24 02:59:01] ppdet.data.transform.operators WARNING: The actual image height: 2337 is not equal to the height: 841.0 in annotation, and update sample['h'] by actual image height.
[05/24 02:59:01] ppdet.data.transform.operators WARNING: The actual image width: 1653 is not equal to the width: 595.0 in annotation, and update sample['w'] by actual image width.
[05/24 02:59:01] ppdet.data.transform.operators WARNING: The actual image height: 2337 is not equal to the height: 841.0 in annotation, and update sample['h'] by actual image height.
[05/24 02:59:01] ppdet.data.transform.operators WARNING: The actual image width: 1653 is not equal to the width: 595.0 in annotation, and update sample['w'] by actual image width.
[05/24 02:59:02] ppdet.data.transform.operators WARNING: The actual image height: 2337 is not equal to the height: 841.0 in annotation, and update sample['h'] by actual image height.
[05/24 02:59:02] ppdet.data.transform.operators WARNING: The actual image width: 1653 is no

## Eval

In [ ]:

command = (
    "python3 ../tools/eval.py",
    "-c ../configs/picodet/legacy_model/application/layout_analysis/krcase_layout_train_2_5.yml",
    # "--slim_config ../configs/picodet/legacy_model/application/layout_analysis/krcase_layout_distill.yml",
    # "-o weights=../output/best_model",
    "-o weights=./output/best_model",
    # "use_gpu=true",
    # "--use_vdl=True --vdl_log_dir=vdl_log"
    )
command = " ".join(command)
print(command)

In [ ]:
!{command}

### Export Model

In [ ]:
model_outpath = './exported_model'
weight2use = '20240515'

command = (
    "python3 ../tools/export_model.py",
    f"-c {configfile}",
    # f"--slim_config {slim_configfile}",
    f"-o weights=./trained_weights/output_{weight2use}/best_model/model.pdparams",
    f"--output_dir={model_outpath}"
    # f"-o export.benchmark=True"
    )
command = " ".join(command)
print(command)

In [ ]:
!{command}

## Test 
##### Strategy: use command line method to test trained weights; create inference model for production.

#### Test inference (CLI, using trained weights, w/ vsualization)

In [24]:
# configfile = "../configs/picodet/legacy_model/application/layout_analysis/krcase_layout_train_2_5.yml"
configfile = "../configs/picodet/legacy_model/application/layout_analysis/krcase_layout_train_tune_2_5.yml"
# slim_configfile = " ../configs/picodet/legacy_model/application/layout_analysis/krcase_layout_distill.yml"

inference_datafolder = './testdata'
# testdoc_img_folder = 'adobe_대법원_2020도1007_판결서'
testdoc_img_folder = 'CONVERTED_JPEG_batch2'
# testdoc_img_folder = 'JPEG_SAMPLE' #'CONVERTED_JPEG'
weight2use = '20240524'
output_dir = './testresults'
output_path = f'{output_dir}/{weight2use}/{testdoc_img_folder}'

if not os.path.exists(output_path):
    os.makedirs(output_path)

command = (
    "python3 ../tools/infer.py",
    f"-c {configfile}",
    # f"--slim_config {slim_configfile}",
    f"--infer_dir={inference_datafolder}/{testdoc_img_folder}",
    # f"--infer_img={inference_datafolder}/{testdoc_img_folder}/11.jpg",
    f"-o weights=./trained_weights/output_{weight2use}/best_model/model.pdparams",
    "--draw_threshold=0.5",
    # "use_gpu=true",
    # "--use_vdl=True",
    # "--vdl_log_dir=./vdl_log_test",
    f"--output_dir={output_path}",
    "--save_results=True",
    "--visualize=True"
    )
command = " ".join(command)
print(command)

python3 ../tools/infer.py -c ../configs/picodet/legacy_model/application/layout_analysis/krcase_layout_train_tune_2_5.yml --infer_dir=./testdata/CONVERTED_JPEG_batch2 -o weights=./trained_weights/output_20240524/best_model/model.pdparams --draw_threshold=0.5 --output_dir=./testresults/20240524/CONVERTED_JPEG_batch2 --save_results=True --visualize=True


In [25]:
!{command}

/bin/bash: /home/lstm/miniconda3/envs/pp261/lib/libtinfo.so.6: no version information available (required by /bin/bash)
W0524 11:20:47.526130 936832 gpu_resources.cc:119] Please NOTE: device: 0, GPU Compute Capability: 7.5, Driver API Version: 12.4, Runtime API Version: 11.8
W0524 11:20:47.526927 936832 gpu_resources.cc:164] device: 0, cuDNN Version: 8.8.
[05/24 11:20:47] ppdet.utils.checkpoint INFO: Finish loading model weights: ./trained_weights/output_20240524/best_model/model.pdparams
[05/24 11:20:47] train INFO: Found 6954 inference images in total.
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
100%|███████████████████████████████████████| 6954/6954 [07:19<00:00, 15.83it/s]
[05/24 11:28:14] ppdet.metrics.metrics INFO: The bbox result is saved to bbox.json.
[05/24 11:28:14] ppdet.metrics.metrics INFO: The bbox result is saved to ./testresults/20240524/CONVERTED_JP

#### Test inference (CLI, using trained weights, bbox only)

In [ ]:
configfile = "../configs/picodet/legacy_model/application/layout_analysis/krcase_layout_train.yml"
# slim_configfile = " ../configs/picodet/legacy_model/application/layout_analysis/krcase_layout_distill.yml"

inference_datafolder = './testdata'
# testdoc_img_folder = 'adobe_대법원_2020도1007_판결서'
# testdoc_img_folder = 'CONVERTED_PNG'
testdoc_img_folder = 'CONVERTED_JPEG'
weight2use = '20240503'
output_dir = './testresults'
output_path = f'{output_dir}/{weight2use}/{testdoc_img_folder}'

if not os.path.exists(output_path):
    os.makedirs(output_path)

command = (
    "python3 ../tools/infer.py",
    f"-c {configfile}",
    # f"--slim_config {slim_configfile}",
    # f"--infer_dir={inference_datafolder}/{testdoc_img_folder}",
    f"--infer_img={inference_datafolder}/{testdoc_img_folder}/11.jpg",
    f"-o weights=./trained_weights/output_{weight2use}/best_model/model.pdparams",
    "--draw_threshold=0.5",
    # "use_gpu=true",
    # "--use_vdl=True",
    # "--vdl_log_dir=./vdl_log_test",
    f"--output_dir={output_path}",
    "--save_results=True",
    "--visualize=False"
    )
command = " ".join(command)
print(command)

In [ ]:
# command = 'python3 ../tools/infer.py -c ../configs/picodet/legacy_model/application/layout_analysis/krcase_layout_train.yml --infer_dir=./testdata/CONVERTED_JPEG -o weights=./trained_weights/output_20240503/best_model/model.pdparams --draw_threshold=0.5 --output_dir=./testresults/20240503/CONVERTED_JPEG'

import subprocess
results = subprocess.run(command, shell=True, capture_output=True, text=True)
jsonpath = results.stdout.split("bbox result is saved to ")[-1].split('.json')[0] + '.json'

with open(jsonpath) as f:
  bboxes = json.load(f)

In [ ]:
# run visualdl in shell (from ./tuning dir)
# visualdl --logdir ./vdl_log_test --port 3001

#### Test inference (using exported model with ../deploy/python/infer.py)
- SEE: https://github.com/PaddlePaddle/PaddleDetection/blob/release/2.7/deploy/python/README.md

### Parameters

| parameter         | Whether it has to be | meaning                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            |
|-------------------|----------------------|--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| --model_dir       | Yes                  | The path to the exported model described above                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    |
| --image_file      | Option               | Pictures that need to be predicted                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                |
| --image_dir       | Option               | The path to the picture folder to be predicted                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    |
| --video_file      | Option               | Videos that need to be predicted                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| --camera_id       | Option               | The ID of the camera used for prediction is -1 by default (it means that no camera prediction is used, it can be set to: 0 - (number of cameras - 1)), and the prediction result is output to output/output.mp4 in the visualization interface`q`                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| --device          | Option               | The device at runtime, optional, defaults`CPU/GPU/XPUCPU`                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         |
| --run_mode        | Option               | When using the GPU, the default is paddle, optional (paddle/trt_fp32/trt_fp16/trt_int8)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           |
| --batch_size      | Option               | The batch size at the time of prediction is valid when specified, and the default value is 1`image_dir`                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            |
| --threshold       | Option               | The threshold for the prediction score, which defaults to 0.5                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
| --output_dir      | Option               | The root directory where the visualization results are saved, defaults to output/                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 |
| --run_benchmark   | Option               | Whether to run benchmark or not, which is False by default`--image_file--image_dir`                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                |
| --enable_mkldnn   | Option               | Whether MKLDNN acceleration is enabled in the CPU prediction is False by default                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| --cpu_threads     | Option               | Set the number of CPU threads, which is 1 by default                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              |
| --trt_calib_mode  | Option               | Whether TensorRT uses the calibration function is false by default. This parameter must be set to True when using the int8 function of TensorRT and False when using PaddleSlim for quantized models                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 |
| --save_images     | Option               | Whether to save the visualization                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 |
| --save_results    | Option               | Whether to save the prediction results of the image in the form of JSON in the folder                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              |

In [ ]:
datapath = './testdata/JPEG_SAMPLE'
filelist = os.listdir(datapath)
print(filelist)

In [ ]:
configfile = "../configs/picodet/legacy_model/application/layout_analysis/krcase_layout_train.yml"
# slim_configfile = " ../configs/picodet/legacy_model/application/layout_analysis/krcase_layout_distill.yml"
# opts = "weights=./trained_weights/output_20240423/best_model/model.pdparams"

inference_datafolder = './testdata/JPEG_SAMPLE'
# testdoc_img_folder = 'adobe_대법원_2020도1007_판결서'
imgname = filelist[0]

inference_outputfolder = './testoutput'
# testout_img_folder = 'adobe_대법원_2020도1007_판결서'

model_path = './exported_model/krcase_layout_train_20240503/'

In [ ]:
command = (
    "python3 ../deploy/python/infer.py",
    f"--model_dir={model_path}",
    f"--image_file={inference_datafolder}/{imgname}",
    "--device=GPU"
    )
command = " ".join(command)
print(command)


In [ ]:
!{command}


In [ ]:
command

# <u>TBD FROM HERE

## Deploy model (use PaddleOCR)

#### Model export (LCNet 2.5) 
https://github.com/PaddlePaddle/PaddleOCR/blob/release/2.7/ppstructure/docs/quickstart_en.md

#### Model inference (distilled model)

| Field | Description | Default |
|-|-|-|  
| output | result save path | ./output/table |
| table_max_len | long side of the image resize in table structure model | 488 |
| table_model_dir | Table structure model inference model path | None |
| table_char_dict_path | The dictionary path of table structure model | ../ppocr/utils/dict/table_structure_dict.txt |
| merge_no_span_structure | In the table recognition model, whether to merge '<td>' and '</td>' | False |
| layout_model_dir | Layout analysis model inference model path | None |
| layout_dict_path | The dictionary path of layout analysis model | ../ppocr/utils/dict/layout_publaynet_dict.txt |
| layout_score_threshold | The box threshold path of layout analysis model | 0.5 |
| layout_nms_threshold | The nms threshold path of layout analysis model | 0.5 |
| kie_algorithm | kie model algorithm | LayoutXLM |
| ser_model_dir | Ser model inference model path | None |
| ser_dict_path | The dictionary path of Ser model | ../train_data/XFUND/class_list_xfun.txt |
| mode | structure or kie | structure |
| image_orientation | Whether to perform image orientation classification in forward | False |
| layout | Whether to perform layout analysis in forward | True |
| table | Whether to perform table recognition in forward | True |
| ocr | Whether to perform ocr for non-table areas in layout analysis. When layout is False, it will be automatically set to False | True |
| recovery | Whether to perform layout recovery in forward | False |
| save_pdf | Whether to convert docx to pdf when recovery | False |
| structure_version | Structure version, optional PP-structure and PP-structurev2 | PP-structure |


## Inference with deployed model (Python)

#### using PPStructure class

In [ ]:
# TODO

import os
import cv2
import matplotlib.pyplot as plt
from paddleocr import PPStructure,draw_structure_result,save_structure_res
from paddleocr import PaddleOCR

from PIL import Image
from IPython.display import display


inference_datafolder = './testdata/CONVERTED_JPEG'
testdoc_img_folder = 'CONVERTED_JPEG'

inference_outputfolder = './testresults_deployment'
testout_img_folder = 'CONVERTED_JPEG'

font_path = '../NotoSansKR-Regular.otf' # PaddleOCR

imgs = os.listdir(f"{inference_datafolder}")
imgs


In [ ]:
# TODO

# get layout
# layout_engine = PPStructure(table=False, ocr=False, show_log=True) #, image_orientation=True)

run_OCR = True #

layout_engine = PPStructure(
    lang='korean',
    output=inference_outputfolder,
    merge_no_span_structure=False,
    # [PaddleOCR] ocr_version
    # [PaddleOCR]use_onnx
    det_model_dir=None,
    # det_algorithm #[DB]
    rec_model_dir=None,
    # [PaddleOCR] rec_algorithm #["CRNN", "SVTR_LCNet"]
    rec_char_dict_path=None, # Path(__file__).parent / rec_model_config["dict_path"]
    # [PaddleOCR] cls_model_dir
    # [PaddleOCR] rec_image_shape # if ocr_version in ["PP-OCRv3", "PP-OCRv4"]
    table_model_dir=None,
    table_max_len=None, # 488
    table_char_dict_path=None,
    layout_model_dir="./exported_model/krcase_layout_train_??", #None
    # layout_model_dir="./exported_model/picodet_lcnet_x1_0_fgd_layout_infer",
    # layout_model_dir="./exported_model/krcase_layout_train_20240503",
    layout_dict_path=None, # use <layout_dict_path> ? (dict should be in exported model)
    layout_score_threshold=0.4, # 0.5
    layout_nms_threshold= 0.3, # 0.5
    kie_algorithm=None, # LayoutXLM
    ser_model_dir=None,
    ser_dict_path=None, #../train_data/XFUND/class_list_xfun.txt
    mode="ocr", #structure", # ocr
    image_orientation=False,
    layout=True,
    table=False,
    ocr=run_OCR,
    recovery=False,
    save_pdf=False,
    structure_version="PP-StructureV2", #"PP-Structure"
    show_log=True
    )

for imgpath in imgs:

    img = cv2.imread(f"{inference_datafolder}/{imgpath}")
    # display(img)
    result = layout_engine(img)
    save_structure_res(result, inference_outputfolder, os.path.basename(imgpath).split('.')[0])

    im_show = draw_structure_result(img, result,font_path=font_path)
    imgbyte = Image.fromarray(im_show)

    # create output folder if not existing
    if not os.path.isdir(f"{inference_outputfolder}"):
        os.mkdir(f"{inference_outputfolder}")


    # show left half only (if no OCR is run)
    if not run_OCR:
        height, width = im_show.shape[:2]
        mid_x = width // 2

        # Crop left side
        box = (0, 0, mid_x, height)
        img_left = imgbyte.crop(box)

        img_left.save(f"{inference_outputfolder}/{imgpath}")

        # display image
        display(img_left)
    else:
        imgbyte.save(f"{inference_outputfolder}/{imgpath}")

        # display image
        display(imgbyte)



    if imgpath == imgs[0]:
        break

    # diplay image with matplotlib without axis (from im_show ndarray)
    # dpi = 200
    # height, width, depth = im_show.shape
    # figsize = width / float(dpi), height / float(dpi)
    # fig = plt.figure(figsize=figsize, dpi=dpi)
    # ax = fig.add_axes([0, 0, 1, 1])
    # ax.axis('off') # hide axis
    # plt.imshow(im_show)